In [40]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer


In [41]:
df = pd.read_csv('archive/all_symptoms.csv')
df = pd.read_csv('archive/dataset_Diseases_and_Symptoms.csv')
#Data proces
all_symptoms = df.columns[1:]
allsss = pd.DataFrame(all_symptoms)

# Write the DataFrame to a CSV file
allsss.to_csv('datatest2.csv', index=True)

# unique_first_field_values = df.iloc[:, 0].drop_duplicates()
# # Transpose
# unique_first_field_values_df = pd.DataFrame(unique_first_field_values).T


In [42]:
df2 = pd.read_csv('datatest2.csv')

In [38]:
df2.head()

,Unnamed: 0,0
0,0,anxiety and nervousness
1,1,depression
2,2,shortness of breath
3,3,depressive or psychotic symptoms
4,4,sharp chest pain


In [35]:
all_symptoms.head()

AttributeError: 'Index' object has no attribute 'head'

In [30]:
all_symptoms

,diseases
0,anxiety and nervousness
1,depression
2,shortness of breath
3,depressive or psychotic symptoms
4,sharp chest pain
...,...
372,hip weakness
373,back swelling
374,ankle stiffness or tightness
375,ankle weakness


In [43]:
embedModel = SentenceTransformer("pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")

def get_embedding(text):
    return embedModel.encode(text)



In [45]:
import faiss


In [46]:
# Generate embeddings
symptom_vectors = np.array([get_embedding(s) for s in all_symptoms]).astype('float32')

# Store using FAISS


In [ ]:
dim = symptom_vectors.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(symptom_vectors)

# Map vector index to original symptom
symptom_map = {i: all_symptoms[i] for i in range(len(all_symptoms))}

In [47]:
def find_similar_symptoms(user_input, top_k=5):
    query_vec = get_embedding(user_input).astype('float32').reshape(1, -1)
    distances, indices = index.search(query_vec, top_k)
    # Filter out any indices that are -1
    threshold = 1.0  # tune this based on your needs
    valid_results = [(symptom_map[i], distances[0][j]) for j, i in enumerate(indices[0]) if i != -1]
    return valid_results


In [48]:
user_input = "anxiety"
results = find_similar_symptoms(user_input)

print("I found these matching symptoms:")
for symptom, score in results:
    print(f"- {symptom} (distance: {score:.4f})")

I found these matching symptoms:
- anxiety and nervousness (distance: 92.2544)
- emotional symptoms (distance: 128.6182)
- fears and phobias (distance: 155.5813)
- depression (distance: 163.0554)
- excessive anger (distance: 190.3128)


In [6]:
user_symptoms_input = [results]

# Extract just the symptom names
user_symptoms_input = [symptom[0] for symptom in user_symptoms_input[0]]
# Result
print(user_symptoms_input)

['nasal congestion', 'headache', 'dizziness', 'symptoms of the face', 'nausea']


In [7]:
user_symptom_vector = [1 if symptom in user_symptoms_input else 0 for symptom in all_symptoms]

In [4]:
X = df.iloc[:,1:378]
X.head()

,anxiety and nervousness,depression,shortness of breath,depressive or psychotic symptoms,sharp chest pain,dizziness,insomnia,abnormal involuntary movements,chest tightness,palpitations,...,stuttering or stammering,problems with orgasm,nose deformity,lump over jaw,sore in nose,hip weakness,back swelling,ankle stiffness or tightness,ankle weakness,neck weakness
0,1,0,1,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y = df.iloc[0:,0]
y.head()

0    panic disorder
1    panic disorder
2    panic disorder
3    panic disorder
4    panic disorder
Name: diseases, dtype: object

In [7]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,
    max_depth=25,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42)
rf.fit(X,y)

RandomForestClassifier(class_weight='balanced', max_depth=25, random_state=42)

In [7]:
import joblib

In [8]:
rf = joblib.load( 's2d-v1.pkl')


In [9]:
user_symptoms_input = ['anxiety and nervousness', 'chest tightness', 'depressive or psychotic symptoms', 'shortness of breath', 'breathing fast', 'irregular heartbeat']
user_symptom_vector = [1 if symptom in user_symptoms_input else 0 for symptom in all_symptoms]
# input_df = pd.DataFrame([user_symptom_vector], columns=all_symptoms)


In [10]:
probs = rf.predict_proba([user_symptom_vector])[0]
top_indices = probs.argsort()[-3:][::-1]
top_diagnoses = [(rf.classes_[i], probs[i]) for i in top_indices]
top_diagnoses

C:\Users\P14s Gen 5\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but RandomForestClassifier is expecting 377 features as input.

MemoryError: Unable to allocate 527. MiB for an array with shape (69065231,) and data type float64